<a href="https://colab.research.google.com/github/annikarichardson2024/Project1/blob/main/SQL_Connect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Find your data sources....saklia, csv of oscars and country API
1.1 Make you new datqbase tables.
2. Connect to the YOUR database aer6bk and make you tables..using the create table command. you can python to do
3. Find your your data and extract and [push it to you new datatbases....SQL Select --> DF --> df.to_sql
4. for csc same...load csv into your DF and then DF.to_sql
5...API call for hte world...put back in the datafarame and to sql...

last thing is to talk to your new database. select statement on aer6bk

In [45]:
!pip install PyMySQL
!pip install mysql-connector-python
!pip install sqlalchemy

In [11]:
host_name = "datatbase.ds2002.org"
#host_ip = "127.0.0.1"
port = "3306"

user_id = "aer6bk"
pwd = "aer6bk!"
db_name = "aer6bk"

In [10]:
import pandas as pd
import os
import pymysql
import warnings

conn = pymysql.connect(host=host_name, user=user_id, password=pwd, database=db_name)
cursor = conn.cursor()

try:
    cursor.execute('SELECT * FROM actor;')

    for row in cursor.fetchmany(size=10):
        print(row)

    cursor.close()

except:
    print ("Error: unable to fetch data")

conn.close()

(1, 'PENELOPE', 'GUINESS', datetime.datetime(2006, 2, 15, 4, 34, 33))
(2, 'NICK', 'WAHLBERG', datetime.datetime(2006, 2, 15, 4, 34, 33))
(3, 'ED', 'CHASE', datetime.datetime(2006, 2, 15, 4, 34, 33))
(4, 'JENNIFER', 'DAVIS', datetime.datetime(2006, 2, 15, 4, 34, 33))
(5, 'JOHNNY', 'LOLLOBRIGIDA', datetime.datetime(2006, 2, 15, 4, 34, 33))
(6, 'BETTE', 'NICHOLSON', datetime.datetime(2006, 2, 15, 4, 34, 33))
(7, 'GRACE', 'MOSTEL', datetime.datetime(2006, 2, 15, 4, 34, 33))
(8, 'MATTHEW', 'JOHANSSON', datetime.datetime(2006, 2, 15, 4, 34, 33))
(9, 'JOE', 'SWANK', datetime.datetime(2006, 2, 15, 4, 34, 33))
(10, 'CHRISTIAN', 'GABLE', datetime.datetime(2006, 2, 15, 4, 34, 33))


In [22]:
# LOAD CSV

df = pd.read_csv('https://raw.githubusercontent.com/annikarichardson2024/Project1/main/Copy%20of%20IMDB%20Top250%20Movies.csv')
df.loc[:,['Movies Names','Rating out of 10'] ]

,Movies Names,Rating out of 10
0,The Shawshank Redemption,9.3
1,The Godfather,9.2
2,The Dark Knight,9.0
3,The Godfather: Part II,9.0
4,12 Angry Men,9.0
...,...,...
245,The Help,8.1
246,It Happened One Night,8.1
247,Aladdin,8.0
248,Dances with Wolves,8.0


In [34]:
conn = pymysql.connect(host=host_name, user=user_id, password=pwd, database=db_name)
cursor = conn.cursor()

df.to_sql(con=conn, name='Movies', if_exists='replace', flavor='mysql')

TypeError: NDFrame.to_sql() got an unexpected keyword argument 'flavor'

In [35]:
from sqlalchemy import create_engine
from sqlalchemy import text

db_name='aer6bk'
pwd = 'aer6bk!'
user_id='aer6bk'

def insert_sqlalchemy_dataframe(user_id, pwd, host_name, db_name, df, table_name):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    df.to_sql(table_name, con=connection,if_exists='append')
    connection.close()

In [40]:
def get_sqlalchemy_dataframe(user_id, pwd, host_name, db_name, sql_query):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    dframe = pd.read_sql(sql_query, connection);
    connection.close()

    return dframe

In [41]:
insert_sqlalchemy_dataframe(user_id, pwd, host_name, db_name, df, 'Employees');

In [44]:
sql_query ="Select * from Employees;"
df = get_sqlalchemy_dataframe(user_id, pwd, host_name, db_name, text(sql_query))
df.head(5)

,index,Movies Names,Rating out of 10,Count of Ratings,Release Date,Budget,Domestic Gross,Domestic Gross2,Domestic Weekend Gross,Domestic Weekend,Domestic Weekend Gross Date,Worldwide Gross
0,0,The Shawshank Redemption,9.3,2.9M,"October 14, 1994 (India)","$25,000,000 (estimated)",US & Canada,$28767189,US & Canada,$727327,Sep 25 1994,"$28,904,642"
1,1,The Godfather,9.2,2M,1978 (India),"$6,000,000 (estimated)",US & Canada,$136381073,US & Canada,$302393,Mar 19 1972,"$250,342,030"
2,2,The Dark Knight,9.0,2.9M,"July 18, 2008 (India)","$185,000,000 (estimated)",US & Canada,$534987076,US & Canada,$158411483,Jul 20 2008,"$1,008,486,720"
3,3,The Godfather: Part II,9.0,1.4M,1979 (India),"$13,000,000 (estimated)",US & Canada,$47834595,US & Canada,$171417,Nov 10 2019,"$47,962,897"
4,4,12 Angry Men,9.0,861K,"April 10, 1957 (United States)","$350,000 (estimated)",US & Canada,Not Found,US & Canada,Not Found,Not Found,"$2,667"


In [14]:
# API CALL
import os
import json
import pprint
import requests
import requests.exceptions
import pandas as pd


def get_api_response(url, response_type):
    try:
        response = requests.get(url)
        response.raise_for_status()

    except requests.exceptions.HTTPError as errh:
        return "An Http Error occurred: " + repr(errh)
    except requests.exceptions.ConnectionError as errc:
        return "An Error Connecting to the API occurred: " + repr(errc)
    except requests.exceptions.Timeout as errt:
        return "A Timeout Error occurred: " + repr(errt)
    except requests.exceptions.RequestException as err:
        return "An Unknown Error occurred: " + repr(err)

    if response_type == 'json':
        result = json.dumps(response.json(), sort_keys=True, indent=4)
    elif response_type == 'dataframe':
        result = pd.json_normalize(response.json())
    else:
        result = "An unhandled error has occurred!"

    return result


In [19]:
url = "https://restcountries.com/v3.1/name/de"
response_type = ['json', 'dataframe']
df = get_api_response(url, response_type[1])
print(df.shape)
print(df.columns)
df.info()



(68, 361)
Index(['tld', 'cca2', 'ccn3', 'cca3', 'cioc', 'independent', 'status',
       'unMember', 'capital', 'altSpellings',
       ...
       'name.nativeName.tam.common', 'currencies.LKR.name',
       'currencies.LKR.symbol', 'languages.sin', 'languages.tam',
       'name.nativeName.rus.official', 'name.nativeName.rus.common',
       'currencies.RUB.name', 'currencies.RUB.symbol', 'languages.rus'],
      dtype='object', length=361)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68 entries, 0 to 67
Columns: 361 entries, tld to languages.rus
dtypes: bool(3), float64(13), int64(1), object(344)
memory usage: 190.5+ KB


In [ ]:
json_string = get_api_response(url, response_type[0])
print(json_string)